In [48]:
import pandas as pd
import numpy as np

In [49]:
df = pd.read_csv(r'C:\Users\hp\Documents\Datasets\R\books.csv')

In [50]:
df.head()

,isbn13,isbn10,title,subtitle,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count
0,9780002005883,0002005883,Gilead,NaN,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0
1,9780002261982,0002261987,Spider's Web,A Novel,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0
2,9780006163831,0006163831,The One Tree,NaN,Stephen R. Donaldson,American fiction,http://books.google.com/books/content?id=OmQaw...,Volume Two of Stephen Donaldson's acclaimed se...,1982.0,3.97,479.0,172.0
3,9780006178736,0006178731,Rage of angels,NaN,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0
4,9780006280897,0006280897,The Four Loves,NaN,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0


In [51]:
df1 = df.copy()

In [52]:
df.drop(columns=['isbn13', 'isbn10', 'subtitle', 'thumbnail', 'published_year', 'num_pages', 'ratings_count'], axis=1, inplace=True)

In [53]:
df.head()

,title,authors,categories,description,average_rating
0,Gilead,Marilynne Robinson,Fiction,A NOVEL THAT READERS and critics have been eag...,3.85
1,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,A new 'Christie for Christmas' -- a full-lengt...,3.83
2,The One Tree,Stephen R. Donaldson,American fiction,Volume Two of Stephen Donaldson's acclaimed se...,3.97
3,Rage of angels,Sidney Sheldon,Fiction,"A memorable, mesmerizing heroine Jennifer -- b...",3.93
4,The Four Loves,Clive Staples Lewis,Christian life,Lewis' work on the nature of love divides love...,4.15


In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6810 entries, 0 to 6809
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           6810 non-null   object 
 1   authors         6738 non-null   object 
 2   categories      6711 non-null   object 
 3   description     6548 non-null   object 
 4   average_rating  6767 non-null   float64
dtypes: float64(1), object(4)
memory usage: 266.1+ KB


In [55]:
df.dropna(inplace=True)

In [56]:
df.duplicated().sum()

4

In [57]:
df.drop_duplicates(inplace=True)

In [58]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6408 entries, 0 to 6809
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           6408 non-null   object 
 1   authors         6408 non-null   object 
 2   categories      6408 non-null   object 
 3   description     6408 non-null   object 
 4   average_rating  6408 non-null   float64
dtypes: float64(1), object(4)
memory usage: 300.4+ KB


In [59]:
df.fillna('', inplace=True)

### Content Based Filtering

In [61]:
df['combined'] = df['title'] + ' ' + df['authors'] + ' ' + df['categories'] + ' ' + df['description']

In [62]:
df['combined']

0       Gilead Marilynne Robinson Fiction A NOVEL THAT...
1       Spider's Web Charles Osborne;Agatha Christie D...
2       The One Tree Stephen R. Donaldson American fic...
3       Rage of angels Sidney Sheldon Fiction A memora...
4       The Four Loves Clive Staples Lewis Christian l...
                              ...                        
6803    Journey to the East Hermann Hesse Adventure st...
6804    The Monk Who Sold His Ferrari: A Fable About F...
6805    I Am that Sri Nisargadatta Maharaj;Sudhakar S....
6808    The Berlin Phenomenology Georg Wilhelm Friedri...
6809    'I'm Telling You Stories' Helena Grice;Tim Woo...
Name: combined, Length: 6408, dtype: object

In [63]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['combined'])

In [64]:
from sklearn.neighbors import NearestNeighbors

model = NearestNeighbors(metric='cosine', algorithm='brute')
model.fit(tfidf_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [65]:
def user_like(title, n=5):
    index = df[df['title'] == title].index[0]
    distances, indices = model.kneighbors(tfidf_matrix[index], n_neighbors=n+1)

    print(f"Books similar to '{title}':")
    for i in range(1, len(indices[0])):
        print(df.iloc[indices[0][i]]['title'])

In [73]:
user_like('The One Tree')

Books similar to 'The One Tree':
A Dark and Hungry God Arises
Reave the Just and Other Tales
Stephen King Omnibus
Forbidden Knowledge
The Gap Into Madness


### Collaborative Filtering

In [75]:
df3 = pd.read_csv(r'C:\Users\hp\Documents\Datasets\movie lens\movies.csv')
df4 = pd.read_csv(r'C:\Users\hp\Documents\Datasets\movie lens\ratings.csv')

In [91]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


In [101]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [103]:
df4_unique = df4.drop_duplicates(subset=['movieId'])
merged = pd.merge(df3, df4_unique, how='inner', on='movieId')

In [107]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9724 entries, 0 to 9723
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   movieId    9724 non-null   int64  
 1   title      9724 non-null   object 
 2   genres     9724 non-null   object 
 3   userId     9724 non-null   int64  
 4   rating     9724 non-null   float64
 5   timestamp  9724 non-null   int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 455.9+ KB


In [145]:
ratings_matrix = merged.pivot_table(index='userId', columns='title', values='rating')
ratings_matrix.fillna(0, inplace=True)

# Transpose for item-based filtering
item_user_matrix = ratings_matrix.T

In [147]:
item_user_matrix.head()

userId,1,2,3,4,5,6,7,8,9,10,...,600,601,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
'Hellboy': The Seeds of Creation (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Round Midnight (1986),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Salem's Lot (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Til There Was You (1997),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [149]:
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix

sparse_matrix = csr_matrix(item_user_matrix.values)

model = NearestNeighbors(metric='cosine', algorithm='brute')
model.fit(sparse_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [155]:
def recommend_collab(movie_title, n=10):
    if movie_title not in item_user_matrix.index:
        print("Movie not found.")
        return

    movie_idx = item_user_matrix.index.get_loc(movie_title)
    distances, indices = model.kneighbors(sparse_matrix[movie_idx], n_neighbors=n+1)

    print(f"\nMovies similar to '{movie_title}':")
    for i in range(1, len(indices[0])):
        print(item_user_matrix.index[indices[0][i]])

In [157]:
recommend_collab('(500) Days of Summer (2009)')


Movies similar to '(500) Days of Summer (2009)':
Ronin (1998)
U-571 (2000)
Frequency (2000)
Edge of Tomorrow (2014)
28 Days Later (2002)
Iron Man (2008)
Lethal Weapon 2 (1989)
Limitless (2011)
Sully (2016)
I, Robot (2004)
